In [1]:
from model import *

byte_training/tok.json
loaded a model with  4201 vocabulary!
target vocab size is:  None !
Output shape (context): torch.Size([1, 6, 16])


In [3]:
# Example input
phrases = ['to pimp a ', 'my favorite color is', 'i love to smoke', 'will you marry']
tokens = tokenizer.encode(phrase)
input_tensor = torch.tensor(tokens).unsqueeze(0)  # (1, seq_length)

# Initialize abstract layers
pos_encoding_layer = PositionalEncoding(max_length=max_length, h_dim=h_dim)
encoding_layer = Encoding(embedding_layer, pos_encoding_layer)
d_out = 16
attention_layer = MaskedSelfAttention(d_in=h_dim, d_out=d_out, context_length=max_length)

# Call them in order
embedded_with_pos = encoding_layer(input_tensor)
context = attention_layer(embedded_with_pos)

print("Output shape (context):", context.shape)

class MyStackedFunction(torch.nn.Module):
    def __init__(self, module_cls, N, *args, **kwargs):
        super().__init__()
        self.modules_list = torch.nn.ModuleList([module_cls(*args, **kwargs) for _ in range(N)])

    def forward(self, x):
        outputs = [mod(x) for mod in self.modules_list]
        return torch.stack(outputs, dim=0).squeeze()

num_heads = 12
StackedAttention = MyStackedFunction(
    MaskedSelfAttention,
    num_heads,
    h_dim,
    d_out,
    max_length,
    dropout=0.5,
    qkv_bias=True
)


Output shape (context): torch.Size([1, 6, 16])
